In [1]:
from openai import OpenAI
import os
import logging
from miniautogen.pipeline.pipeline import PipelineComponent, ChatPipelineState
from miniautogen.pipeline.components import (
    NextAgentSelectorComponent,
    Jinja2TemplatesComponent,
    NextAgentMessageComponent,
    UpdateNextAgentComponent,
    LLMResponseComponent
)

In [2]:
from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components import (
    UserResponseComponent,
    AgentReplyComponent,
    TerminateChatComponent,
    NextAgentSelectorComponent,
    UserInputNextAgent
)

PROMPT_TEMPLATE_AGENT_SYSTEM = """
# Introdução
- Você é um agente conforme descrito na seção "SUA FUNÇÃO".
- Você atua em uma conversa colaborativa com uma EQUIPE DE AGENTES, focada em solucionar uma TAREFA específica.

# Tarefa da Equipe
- Objetivo da equipe: {{chat.context['goal']}}

# Sua Função
- NOME DO AGENTE: {{agent.name}}
- DESCRIÇÃO DO AGENTE: 
{{agent.role}}

# Sua Equipe de Agentes
{% for agent in chat.agentList %}
  - {{agent.name}}
{% endfor %}

# Dinâmica da Conversa
- Considere TODAS as mensagens anteriores para construir sua resposta.
- Você é o {{agent.name}}, nunca confunda sua identidade com a de outro agente.
- Identificação do remetente: Cada mensagem terá um "SENDER_ID".

# Instruções
- Mantenha foco na sua função específica.
- Contribua efetivamente para o sucesso da TAREFA.

# Sua Equipe de Agentes
- Aqui estão as descrições e especializações dos membros da equipe:
{% for agent in chat.agentList %}
  - {{agent.name}}
{% endfor %}

# Formato da Resposta
- Responda apenas com o conteúdo da sua mensagem, sem incluir respostas de outros agentes.
- Assegure que sua resposta seja relevante e contribua para o avanço da discussão.
"""


PROMPT_TEMPLATE_USER = """
HISTÓRICO DE CONVERSAÇÃO:
-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~
[
  {% for message in messages %}
    {"sender_id": "{{ message['sender_id'] }}", "message": "{{ message['message'] | escape }}"}{% if not loop.last %}, {% endif %}
  {% endfor %}
]
-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~
"""
from miniautogen.llms.llm_client import OpenAIClient
from dotenv import load_dotenv
load_dotenv()

api_key=os.getenv('OPENAI_API_KEY')
openai_client = OpenAIClient(api_key=api_key)

# Cria uma instância do componente com o template
jinja_component = Jinja2TemplatesComponent()
jinja_component.add_template(PROMPT_TEMPLATE_AGENT_SYSTEM, 'system')
jinja_component.add_template(PROMPT_TEMPLATE_USER, 'user')

UpdateNextAgent = UpdateNextAgentComponent()
UpdateNextAgent.set_next_agent_id("agent_admin")

NextAgentMessage = NextAgentMessageComponent()
NextAgentMessage.set_alternative_next(NextAgentSelectorComponent())

# Configuração dos Pipelines
pipeline_user = Pipeline([UserResponseComponent()])
pipeline_jinja = Pipeline([jinja_component, LLMResponseComponent(openai_client)])
pipeline_admin = Pipeline(
    [NextAgentSelectorComponent(), AgentReplyComponent(), TerminateChatComponent()])




In [3]:
# Setup do ambiente de teste
chat_context = {'goal': 'Desenvolver um componente que salva as mensagens de markdown.'}
chat = Chat()

CONHECIMENTO_PREVIO = """
README DO MINIAUTOGEN:
```README.md
# MiniAutoGen: Biblioteca **leve e flexível** para criar agentes e conversas multi-agentes.

## Sobre o MiniAutoGen

O MiniAutoGen é uma biblioteca open source inovadora, projetada para capacitar aplicações de próxima geração em Modelos de Linguagem de Grande Escala (LLMs) através de conversas multi-agentes. Este framework se destaca por sua estrutura leve e flexível, ideal para desenvolvedores e pesquisadores que buscam explorar e expandir as fronteiras da IA conversacional.

## Por que MiniAutoGen?

### Conversas Multi-Agentes
Capacite conversas envolvendo múltiplos agentes inteligentes, cada um com habilidades distintas, elevando a complexidade e sofisticação das interações.

### Customização de Agentes
Ajuste os agentes para atender a requisitos específicos, adaptando comportamento, reações e padrões de resposta conforme o necessário.

### Flexibilidade e Modularidade
Com o MiniAutoGen, você tem a liberdade de moldar conversações dinâmicas, permitindo iniciativas de diálogo dos agentes, reações automáticas e intervenções humanas quando necessário.

### Coordenação Eficaz entre Agentes
Utilize nosso framework para que os agentes colaborem eficientemente, visando atingir objetivos comuns em um ambiente partilhado.

## Principais Componentes

### Agent
O núcleo de cada conversa, representando um agente individual com habilidades e comportamentos específicos, essencial para interações dinâmicas e autônomas.

### Chat
Gerencia sessões de chat em grupo, assegurando a manutenção eficaz do estado e contexto da conversa, essencial para a continuidade e coesão das interações.

### ChatAdmin
Um elemento-chave para a coordenação do chat em grupo, sincronizando ações e gerenciando a dinâmica da conversa para garantir uma colaboração eficiente.

### Pipeline
Automatiza e organiza as operações dos agentes, promovendo a escalabilidade e a manutenção facilitada do sistema.

## Contribua com o MiniAutoGen

Como um projeto open source, o MiniAutoGen convida entusiastas de IA, desenvolvedores e pesquisadores para contribuir e ajudar a moldar o futuro das conversas multi-agentes. Seu conhecimento e experiência podem ajudar a expandir as capacidades do MiniAutoGen, criando soluções mais robustas e versáteis para a comunidade de desenvolvedores.

### Como Você Pode Contribuir:
- **Desenvolvimento de Novos Recursos:** Ajude a adicionar novas funcionalidades e aprimorar as existentes.
- **Documentação e Tutoriais:** Contribua com documentação clara e tutoriais para facilitar o uso do framework por novos usuários.
- **Testes e Feedback:** Participe testando o framework e fornecendo feedback valioso para melhorias contínuas.
- **Compartilhamento de Ideias e Experiências:** Partilhe suas experiências e ideias para enriquecer a comunidade e impulsionar inovações.

## Comece a Contribuir Hoje

Visite nosso repositório no GitHub para saber mais sobre como você pode se envolver e começar a contribuir. Junte-se a nós nessa jornada emocionante para impulsionar o avanço das conversas multi-agentes no mundo da inteligência artificial!

```
---

MiniAutoGen: Desenvolvendo hoje o futuro das conversas inteligentes.
```

### Arquitetura e Componentes do MiniAutoGen

1. **Arquitetura Modular e Extensível:**
   - O MiniAutoGen é projetado com uma arquitetura modular, permitindo que diferentes funções sejam encapsuladas em componentes distintos. 
   - Essa abordagem facilita a extensão e a personalização do sistema, permitindo aos desenvolvedores adicionar ou modificar componentes conforme necessário.

2. **Componentes do Pipeline:**
   - Cada componente representa uma operação ou um conjunto de operações que podem ser realizadas em uma conversa.
   - Estes componentes são organizados em um "pipeline", onde o processamento de uma conversa é conduzido sequencialmente através de vários componentes.

3. **Padrões de Desenvolvimento:**
   - **Princípio da Responsabilidade Única:** Cada componente é responsável por uma tarefa específica, seguindo o princípio de responsabilidade única.
   - **Abstração e Encapsulamento:** Os componentes são abstrações que ocultam a complexidade do processamento interno, oferecendo uma interface clara para interação com o restante do sistema.
   - **Padrão de Projeto Decorator:** O uso de um pipeline onde componentes podem ser adicionados ou removidos dinamicamente sugere uma implementação semelhante ao padrão Decorator, permitindo a composição de comportamentos em tempo de execução.

4. **Tipos de Componentes:**
   - **UserResponseComponent:** Lida com as entradas dos usuários.
   - **AgentReplyComponent:** Gera respostas dos agentes com base nas entradas processadas.
   - **NextAgentSelectorComponent:** Determina qual agente deve responder em seguida, baseando-se na lógica ou estado da conversa.
   - **TerminateChatComponent:** Avalia condições para encerrar a conversa.
   - **OpenAIChatComponent e OpenAIThreadComponent:** Integram com a API da OpenAI para utilizar modelos de linguagem como agentes na conversa.

5. **Gestão de Estado:**
   - O estado da conversa é gerenciado e passado entre componentes. Isso permite a manutenção do contexto e a continuidade ao longo de uma sessão de chat.

6. **Flexibilidade e Customização:**
   - Os desenvolvedores podem criar componentes personalizados para atender a requisitos específicos, integrando funcionalidades externas ou lógicas de negócios complexas.

### Padrões Arquitetônicos

- **Arquitetura Orientada a Serviços (SOA):** Cada componente pode ser visto como um serviço, com entradas, processamento e saídas claramente definidos.
- **Padrão Pipeline:** A sequência de processamento através de componentes distintos segue o padrão de pipeline, comum em processamento de dados e workflows.

### Conclusão

A arquitetura e os padrões de desenvolvimento do MiniAutoGen refletem uma abordagem moderna e modular para a construção de sistemas de conversação. A ênfase na modularidade, extensibilidade e responsabilidade única de cada componente torna o framework adaptável a uma variedade de cenários de uso, promovendo uma implementação eficiente e manutenível.
```

**Exemplo de components:**
```
from openai import OpenAI
import openai
import os
import logging
from dotenv import load_dotenv
from .pipeline import PipelineComponent
import time

class AgentReplyComponent(PipelineComponent):
    def process(self, state):

        Processa a resposta do agente atual e adiciona essa resposta ao chat em grupo.

        Args:
            state (PipelineState): Estado atual do pipeline.

        Returns:
            PipelineState: Estado atualizado do pipeline.

        # Acessa o estado atual para obter informações necessárias
        agent = state.get_state().get('selected_agent')
        group_chat = state.get_state().get('group_chat')
        if not agent or not group_chat:
            raise ValueError("Agent e GroupChat são necessários para AgentReplyComponent.")
        # Implementação da geração da resposta do agente
        try:
            reply = agent.generate_reply(state)
            print(reply)
            group_chat.add_message(sender_id=agent.agent_id, message=reply)
        except Exception as e:
            print(f"Erro ao processar a resposta do agente: {e}")

        return state
```
"""


INITIAL_MESSAGE = """
Refatorar este component para que fique mais abstrato e possamos utilizar diversos LLMs distintos.
```python`
class OpenAIComponent(PipelineComponent):

    def __init__(self):
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.logger = logging.getLogger(__name__)

    def process(self, state):
        try:
            prompt = state.get_state().get('prompt')
            if not prompt:
                raise ValueError(
                    "groupchat e agent são obrigatórios para OpenAIResponseComponent.")
            response = self._call_openai_api(prompt)
            return response.choices[0].message.content
        except Exception as e:
            self.logger.error(f"Erro em OpenAIResponseComponent: {e}")
            raise

    def _call_openai_api(self, prompt):
         Realiza a chamada à API da OpenAI. 
        try:
            return self.client.chat.completions.create(
                model="gpt-4-1106-preview",
                messages=prompt,
                temperature=1,
            )
        except Exception as e:
            self.logger.error(f"Erro ao chamar a API da OpenAI: {e}")
            raise
```
"""

json_messages = [
    {'sender_id': 'ADMIN', 'message': CONHECIMENTO_PREVIO},
    {'sender_id': 'ADMIN', 'message': INITIAL_MESSAGE}
]

chat.add_messages(json_messages)

In [4]:
PROJECT_MANAGER_SYSTEM_PROMPT = """
Como Agente Gerente de Projeto (Product Owner), você coordena a interface entre os objetivos do projeto e a equipe de desenvolvimento. Suas principais funções são:

1. **Especificação de Requisitos**: Colaborar com a equipe para definir requisitos claros e precisos, assegurando que as expectativas se alinhem com a implementação.
2. **Validação de Código**: Examinar o código produzido para confirmar sua conformidade com os requisitos estabelecidos.
3. **Nível de detalhes**: Fornecer informações e orientações adicionais para garantir que o código seja adequado para o propósito.

VOCE NUNCA DEVE DESENVOLVER O CÓDIGO, APENAS REVISAR E VALIDAR.

Instruções Operacionais:
- Para iniciar o desenvolvimento: Use o comando "DEV_AUTOGEN, POR FAVOR, desenvolva o código para os componentes especificados" após completar a especificação.
- Para concluir as revisões: Emita o comando `TERMINATE` quando o código estiver adequado.
"""



DEV_AUTOGEN_SYSTEM_PROMPT = """
**Tarefa**: Como especialista em desenvolvimento de componentes para a biblioteca MiniAutoGen, crie um componente utilizando Python, com ênfase em técnicas avançadas e melhores práticas de programação. O componente deve estar alinhado com os padrões de design da biblioteca e otimizado para interação e funcionalidade eficientes.

**Habilidades e Conhecimentos Necessários**:
1. **Python Avançado**: Use sua proficiência em Python para aplicar técnicas avançadas e boas práticas de codificação.
2. **Programação Orientada a Objetos (POO)**: Aplique sua expertise em POO para estruturar o componente de forma eficiente e eficaz.
3. **Arquiteturas MVC e SOA**: Incorpore conhecimentos em Model-View-Controller e Service-Oriented Architecture para garantir a organização e modularidade do componente.
4. **Fundamentos de LLMs**: Utilize sua compreensão dos Modelos de Linguagem de Grande Escala, como GPT-3 e GPT-4, para integrar o componente com sistemas de IA conversacional.
5. **Expertise em MiniAutoGen**: Aplique seu conhecimento específico da biblioteca MiniAutoGen para desenvolver uma solução personalizada e eficiente.

**Contexto e Diretrizes**:
1. **Integração com a Arquitetura Existente**: Seu componente deve aderir à arquitetura modular e extensível do MiniAutoGen, respeitando o princípio da responsabilidade única e os padrões de abstração e encapsulamento.
2. **Funcionalidade Específica do Componente**: Escolha uma funcionalidade específica relevante para conversas multi-agentes. Isso pode incluir, mas não está limitado a, gestão de estado, seleção de agentes, terminação de chat, ou integração com modelos de linguagem.
3. **Adesão aos Padrões Arquitetônicos**: Considere a SOA e o padrão de pipeline na construção do seu componente. Ele deve ter entradas, processamento e saídas bem definidos, e ser capaz de se integrar ao fluxo do pipeline existente.
4. **Documentação e Exemplo de Código**: Forneça uma breve documentação explicando a finalidade e o funcionamento do seu componente. Inclua um exemplo de código que demonstre como ele se integra ao MiniAutoGen.

**Informações Adicionais**:
- Utilize as informações fornecidas no README do MiniAutoGen e nos detalhes arquitetônicos como referência.
- Lembre-se de que o MiniAutoGen valoriza a flexibilidade, modularidade e a customização na criação de agentes e conversas multi-agentes.

**Resultado Esperado**:
- Um script Python contendo a implementação do seu componente.
- Documentação associada explicando sua funcionalidade e integração no sistema MiniAutoGen.

**Instruções para a Resposta**:
1. **Código Completo**: Forneça um script Python completo que realize a tarefa.
2. **Salvamento do Código**: Inclua a linha de comentário `# filename: <filename>.py` no início do seu código para indicar o nome do arquivo em que ele deve ser salvo.
TODO SEU DESENVOLVIMENTO DEVE SER REALIZADO SEGUINDO A ESTRUTURA E ARQUITETURA DO MINIAUTOGEN, VEJA OS EXEMPLOS.
"""


agente1_data = {
    'agent_id': 'PM_AUTOGEN',
    'name': 'PM_AUTOGEN',
    'role': PROJECT_MANAGER_SYSTEM_PROMPT}


agente2_data = {
    'agent_id': 'DEV_AUTOGEN',
    'name': 'DEV_AUTOGEN',
    'role': DEV_AUTOGEN_SYSTEM_PROMPT}

agente3_data = {
    'agent_id': 'ADMIN',
    'name': 'ADMIN',
    'role': 'ADMIN'}

# Criação de Agentes
agent1 = Agent.from_json(agente1_data)
agent1.pipeline = pipeline_jinja  # Atribuindo o pipeline ao agente

agent2 = Agent.from_json(agente2_data)
agent2.pipeline = pipeline_jinja  # Atribuindo o pipeline_llm ao segundo agente

agent3 = Agent.from_json(agente3_data)
agent3.pipeline = pipeline_user  # Atribuindo o pipeline_user ao terceiro agente

# Adicionando os agentes ao chat
chat.add_agent(agent1)
chat.add_agent(agent2)
# chat.add_agent(agent3)


# Criação e configuração do ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role",
                       pipeline_admin, chat, "manage_chat", 10)

In [5]:
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1


[{'role': 'system', 'content': '\n# Introdução\n- Você é um agente conforme descrito na seção "SUA FUNÇÃO".\n- Você atua em uma conversa colaborativa com uma EQUIPE DE AGENTES, focada em solucionar uma TAREFA específica.\n\n# Tarefa da Equipe\n- Objetivo da equipe: \n\n# Sua Função\n- NOME DO AGENTE: PM_AUTOGEN\n- DESCRIÇÃO DO AGENTE: \n\nComo Agente Gerente de Projeto (Product Owner), você coordena a interface entre os objetivos do projeto e a equipe de desenvolvimento. Suas principais funções são:\n\n1. **Especificação de Requisitos**: Colaborar com a equipe para definir requisitos claros e precisos, assegurando que as expectativas se alinhem com a implementação.\n2. **Validação de Código**: Examinar o código produzido para confirmar sua conformidade com os requisitos estabelecidos.\n3. **Nível de detalhes**: Fornecer informações e orientações adicionais para garantir que o código seja adequado para o propósito.\n\nVOCE NUNCA DEVE DESENVOLVER O CÓDIGO, APENAS REVISAR E VALIDAR.\n\nIn

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
ERROR:miniautogen.llms.llm_client:Erro ao chamar a API da OpenAI: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 9167 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
ERROR:miniautogen.pipeline.components:Falha ao obter resposta do LLM.
INFO:miniautogen.chat.chatadmin:Executing round 2


Erro ao processar a resposta do agente: ChatPipelineState.update_state() takes 1 positional argument but 2 were given
[{'role': 'system', 'content': '\n# Introdução\n- Você é um agente conforme descrito na seção "SUA FUNÇÃO".\n- Você atua em uma conversa colaborativa com uma EQUIPE DE AGENTES, focada em solucionar uma TAREFA específica.\n\n# Tarefa da Equipe\n- Objetivo da equipe: \n\n# Sua Função\n- NOME DO AGENTE: PM_AUTOGEN\n- DESCRIÇÃO DO AGENTE: \n\nComo Agente Gerente de Projeto (Product Owner), você coordena a interface entre os objetivos do projeto e a equipe de desenvolvimento. Suas principais funções são:\n\n1. **Especificação de Requisitos**: Colaborar com a equipe para definir requisitos claros e precisos, assegurando que as expectativas se alinhem com a implementação.\n2. **Validação de Código**: Examinar o código produzido para confirmar sua conformidade com os requisitos estabelecidos.\n3. **Nível de detalhes**: Fornecer informações e orientações adicionais para garantir

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.879540 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 1.832560 seconds


KeyboardInterrupt: 

In [ ]:
chat.get_messages().to_dict(orient='records')

In [ ]:
chat.get_messages()

In [ ]:
table_md = chat.get_messages()[['sender_id', 'message']]
# Especifique o caminho do arquivo onde deseja salvar o arquivo Markdown
file_path = 'chat.md'

# Abra o arquivo para escrever e salve os registros no formato "Sender_id\nMessage"
with open(file_path, 'w') as file:
    for index, row in table_md.iterrows():
        sender_id = row['sender_id']
        message = row['message']
        
        # Adicionar um cabeçalho com o sender_id em negrito
        file.write(f'### **Sender_id:** {sender_id}\n\n')
        
        # Adicionar uma linha divisória criativa
        file.write('***\n\n')
        
        # Adicionar o conteúdo da mensagem
        file.write(message)
        file.write('\n\n')
        file.write('\n\n')
        file.write('\n\n')

In [ ]:
table_md